In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.transform import resize
import matplotlib.pyplot as plt
from nilearn import plotting

In [ ]:
# Charger l'image à l'aide de nibabel
img = nib.load('/Users/salimrami/Desktop/sub/sub1_T2w.nii.gz')

# Obtenir les données de l'image
data = img.get_fdata()
print(data.shape)
plotting.plot_anat(img)
per = np.percentile(data, [25, 50, 95])


threshold = 90.0 # Par exemple, tronquer les 10% supérieurs de l'intensité des pixels

# Calculer la valeur seuil correspondant au percentile
truncation_value = np.percentile(data, threshold)

# Tronquer les valeurs de pixel supérieures à la valeur seuil
truncated_data = np.where(data > truncation_value, truncation_value, data)

# Créer une nouvelle image NIfTI avec les données tronquées
truncated_image = nib.Nifti1Image(truncated_data, affine=img.affine)


nib.save(truncated_image, '/Users/salimrami/Desktop/sub/tronqué.nii.gz')  # Spécifiez le chemin de sauvegarde souhaité






# Calculer la moyenne d'intensité
moyenne_intensite = np.mean(truncated_data)
max = np.max(truncated_data)

# Afficher la moyenne d'intensité
print("Moyenne d'intensité de l'image :", moyenne_intensite)
print("Max d'intensité de l'image :", max)
# Afficher l'image
plotting.plot_anat(truncated_image)

In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.ndimage import zoom

def contrast_correction(reference_image_data, low_res_image_data):
    # Add a small epsilon value to avoid division by zero
    epsilon = 1e-8
    denominator = reference_image_data + epsilon

    # Perform Gaussian low-pass filtering on the relative scaling factor
    scaling_factor = low_res_image_data / denominator
    filtered_scaling_factor = gaussian_filter(scaling_factor, sigma=1.0)

    # Keep the mean value of the low-resolution image unchanged
    mean_scaling_factor = np.mean(filtered_scaling_factor)
    scaled_low_res_image = low_res_image_data / mean_scaling_factor

    return scaled_low_res_image

# Load the reference image and low-resolution image using nibabel
reference_image_path = "/Users/salimrami/Desktop/dhcp-images6/dhcp-1.nii.gz"
low_res_image_path = "/Users/salimrami/Desktop/sub/tronqué.nii.gz"

reference_image_nifti = nib.load(reference_image_path)
low_res_image_nifti = nib.load(low_res_image_path)

# Get the image data as NumPy arrays
reference_image_data = reference_image_nifti.get_fdata()
low_res_image_data = low_res_image_nifti.get_fdata()

# Calculate the scaling factors along each axis
scaling_factors = [ref_dim / low_dim for ref_dim, low_dim in zip(reference_image_data.shape, low_res_image_data.shape)]

# Resample the low-resolution image to match the dimensions of the reference image
resampled_low_res_image_data = zoom(low_res_image_data, scaling_factors)

# Call the contrast correction function
corrected_low_res_image_data = contrast_correction(reference_image_data, resampled_low_res_image_data)

# Save the corrected image as a new NIfTI file (if needed)
corrected_low_res_nifti = nib.Nifti1Image(corrected_low_res_image_data, reference_image_nifti.affine)
nib.save(corrected_low_res_nifti, "/Users/salimrami/Desktop/sub/finalsub.nii.gz")


In [ ]:
import nibabel as nib
import numpy as np

def normalize_intensity(image_data, new_min, new_max):
    old_min, old_max = np.min(image_data), np.max(image_data)
    normalized_data = (image_data - old_min) * ((new_max - new_min) / (old_max - old_min)) + new_min
    return normalized_data

def main():
    # Chemin du fichier nii.gz
    input_file = '/Users/salimrami/Desktop/sub/finalsub.nii.gz'
    output_file = '/Users/salimrami/Desktop/sub/finalnit.nii.gz'

    # Charger l'image
    img = nib.load(input_file)
    img_data = img.get_fdata()

    # Normaliser les intensités entre 0 et 40
    normalized_data = normalize_intensity(img_data, 0, 40)

    # Créer une nouvelle image NIfTI avec les données normalisées
    normalized_img = nib.Nifti1Image(normalized_data, img.affine, img.header)

    # Sauvegarder l'image normalisée au format nii.gz
    nib.save(normalized_img, output_file)

if __name__ == "__main__":
    main()


In [ ]:
import nibabel as nib

def resample_images_to_reference(ref_img_path, target_img_path, output_img_path):
    # Charger les images NIfTI
    ref_img = nib.load(ref_img_path)
    target_img = nib.load(target_img_path)

    # Récupérer les données des images et les informations d'espacement
    ref_data = ref_img.get_fdata()
    ref_affine = ref_img.affine

    target_data = target_img.get_fdata()
    target_affine = target_img.affine

    # Ramener l'image cible dans le même espace que l'image de référence
    target_resampled = nib.Nifti1Image(target_data, ref_affine)

    # Sauvegarder l'image ramenée dans le même espace
    nib.save(target_resampled, output_img_path)

# Exemple d'utilisation
if __name__ == "__main__":
    ref_img_path = "/Users/salimrami/Desktop/res-final-dhcp-dl/res-0148/sub1_T2w_restore_brain.nii.gz"
    target_img_path = "/Users/salimrami/Desktop/MarsFet/rr-seg.nii.gz"
    output_img_path = "/Users/salimrami/Desktop/MarsFet/rr.nii.gz"

    resample_images_to_reference(ref_img_path, target_img_path, output_img_path)
